In [12]:
# -*- coding: utf-8 -*-
# --------------------------------------------------
#
# CRF5.ipynb
#
# Token features:
#     (a) token (surface form)
#     (b) is_first           : is token at the beginning of the sentence?
#     (c) is_last            : is token at the end of the sentence?
#     (d) is_capitalized     : does token start with a capital letter? 
#     (e) is_all_capitalized : is all letters of the token capitalized?
#     (f) is_capitals_inside : is there any capitalized letter inside the token?
#     (g) is_numeric         : is token numeric?
#     (h) is_numeric_inside  : is numeric characters inside the token?
#     (i) is_alphanumeric    : is token alphanumeric?
#     (j) prefix-1           : first letter of the token
#     (k) suffix-1           : last letter of the token
#     (l) prefix-2           : first two letters of the token
#     (m) suffix-2           : last two letters of the token
#     (n) next-token         : following token
#     (o) prev-token         : preceding token
#
# Written by cetinsamet -*- cetin.samet@metu.edu.tr
# May, 2019
# --------------------------------------------------

In [13]:
from sklearn.model_selection import RandomizedSearchCV
from seqeval.metrics import classification_report
from sklearn.metrics import make_scorer
from sklearn_crfsuite import metrics
from sklearn_crfsuite import CRF
from tqdm import tqdm
import pickle
import scipy

In [14]:
def readFile(filepath):

    text     = []
    sentence = []

    with open(filepath, 'r') as infile:
        for line in infile:
            word, _, _, _ = line.strip().split('\t')

            if word == '<S>':
                text.append(sentence)
                sentence = []
                continue

            sentence.append(line.strip())

    return text

In [15]:
trainText = readFile('data/train.txt')
validText = readFile('data/valid.txt')
testText  = readFile('data/test.txt')

In [16]:
def getFeature(token, token_index, sentence):


    feature = {'token'             : token,
               'is_first'          : token_index == 0,
               'is_last'           : token_index == len(sentence) - 1,
               
               'is_capitalized'    : token[0].upper() == token[0],
               'is_all_capitalized': token.upper() == token,
               'is_capitals_inside': token[1:].lower() != token[1:],
               'is_numeric'        : token.isdigit(),
               'is_numeric_inside' : any([c.isdigit() for c in token]),
               'is_alphanumeric'   : token.isalnum(),
               
               'prefix-1'          : token[0],
               'suffix-1'          : token[-1],
               
               'prefix-2'          : '' if len(token) < 2  else token[:2],
               'suffix-2'          : '' if len(token) < 2  else token[-2:],
               
               'prev-token'        : '' if token_index == 0 else sentence[token_index - 1],
               'next-token'        : '' if token_index == len(sentence) - 1 else sentence[token_index + 1],
              }
    
    return feature

In [29]:
trainFeatures = []
trainLabels   = []

for sentence_ in tqdm(trainText):

    sentence = [token.split('\t')[0] for token in sentence_]
    labels   = [token.split('\t')[-1] for token in sentence_]
    features = []
    for i, word in enumerate(sentence):
        features.append(getFeature(word, i, sentence))
    
    trainFeatures.append(features)
    trainLabels.append(labels)

100%|██████████| 25736/25736 [00:01<00:00, 15675.58it/s]


In [32]:
validFeatures = []
validLabels   = []

for sentence_ in tqdm(validText):
    
    sentence = [token.split('\t')[0] for token in sentence_]
    labels   = [token.split('\t')[-1] for token in sentence_]
    features = []
    for i, word in enumerate(sentence):
        features.append(getFeature(word, i, sentence))
    
    validFeatures.append(features)
    validLabels.append(labels)

100%|██████████| 6435/6435 [00:00<00:00, 11772.84it/s]


In [33]:
testFeatures = []
testLabels   = []

for sentence_ in tqdm(testText):
    
    sentence = [token.split('\t')[0] for token in sentence_]
    labels   = [token.split('\t')[-1] for token in sentence_]
    features = []
    for i, word in enumerate(sentence):
        features.append(getFeature(word, i, sentence))
    
    testFeatures.append(features)
    testLabels.append(labels)

100%|██████████| 3328/3328 [00:00<00:00, 15746.25it/s]


In [35]:
trainvalFeatures = trainFeatures + validFeatures
trainvalLabels   = trainLabels   + validLabels

In [36]:
# define fixed parameters and parameters to search
crf = CRF(  algorithm='lbfgs',
            max_iterations=100,
            all_possible_transitions=True,
            verbose=True)

params_space = {'c1': scipy.stats.expon(scale=0.5),
                'c2': scipy.stats.expon(scale=0.05)}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=30,
                        random_state=123,
                        scoring=f1_scorer)

rs.fit(trainvalFeatures, trainvalLabels)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/Users/cetinsamet/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 165.3min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 282.0min finished
loading training data to CRFsuite: 100%|██████████| 32171/32171 [00:15<00:00, 2047.61it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 237977
Seconds required: 4.020

L-BFGS optimization
c1: 0.048313
c2: 0.028432
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=2.40  loss=694456.75 active=237701 feature_norm=1.00
Iter 2   time=2.45  loss=662949.65 active=236776 feature_norm=7.12
Iter 3   time=1.24  loss=530738.94 active=228760 feature_norm=6.09
Iter 4   time=10.08 loss=234480.97 active=230998 feature_norm=3.48
Iter 5   time=2.46  loss=221807.04 active=234870 feature_norm=5.28
Iter 6   time=1.27  loss=213065.39 active=235879 feature_norm=5.98
Iter 7   time=1.23  loss=168484.02 active=236658 feature_norm=6.87
Iter 8   time=1.40  loss=158983.02 active=237344 feature_norm=7.32
Iter 9   time=1.39  loss=153452.75 active=237440 feature_norm=

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=None, c2=None,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error...ne,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True),
          fit_params=None, iid='warn', n_iter=30, n_jobs=-1,
          param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1af53e2198>, 'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1af53e2400>},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn',
          scoring=make_scorer(flat_f1_score, average=weighted), verbose=1)

In [37]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.04831323984330459, 'c2': 0.028431668831944812}
best CV score: 0.9686005027337169
model size: 2.83M


In [38]:
crf = rs.best_estimator_
crf

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None,
  c1=0.04831323984330459, c2=0.028431668831944812,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=True)

In [39]:
#crf.fit(trainvalFeatures, trainvalLabels)

In [40]:
# SAVE CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf05.pickle', 'wb') as outfile:
    pickle.dump(crf, outfile, pickle.HIGHEST_PROTOCOL)
    print("model is saved.")

model is saved.


In [41]:
# LOAD CONDITIONAL RANDOM FIELDS MODEL
with open('model/crf05.pickle', 'rb') as infile:
    crf = pickle.load(infile)

In [42]:
trainvalPredLabels = crf.predict(trainvalFeatures)

print("### TRAINVAL CLASSIFICATION REPORT ###\n")
print(classification_report(trainvalLabels, trainvalPredLabels))

### TRAINVAL CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

    LOCATION       0.99      1.00      1.00      9409
        DATE       1.00      1.00      1.00      3103
      PERSON       0.99      0.99      0.99     14476
ORGANIZATION       0.99      0.99      0.99      9034
        TIME       1.00      1.00      1.00       175
     PERCENT       1.00      1.00      1.00       617
       MONEY       1.00      1.00      1.00       594

   micro avg       0.99      0.99      0.99     37408
   macro avg       0.99      0.99      0.99     37408



In [43]:
testPredLabels  = crf.predict(testFeatures)

print("### TEST CLASSIFICATION REPORT ###\n")
print(classification_report(testLabels, testPredLabels))

### TEST CLASSIFICATION REPORT ###

              precision    recall  f1-score   support

    LOCATION       0.87      0.84      0.86      1091
     PERCENT       0.99      0.93      0.96       107
ORGANIZATION       0.82      0.78      0.80       862
      PERSON       0.88      0.84      0.86      1594
       MONEY       0.93      0.76      0.84       113
        TIME       0.94      0.74      0.83        23
        DATE       0.90      0.88      0.89       364

   micro avg       0.87      0.83      0.85      4154
   macro avg       0.87      0.83      0.85      4154

